# Board Game Rules Q&A with RAG and Gemini Flash

Welcome to the **Board Game Rules Q&A** project! This project demonstrates how to build a Retrieval-Augmented Generation (RAG) system for querying board game rule documents. It leverages **ChromaDB** for embedding and document retrieval and the **Gemini Flash API** to generate comprehensive, human-like answers.

---

## Overview

This project performs the following steps:

1. **Indexing:**  
   Index board game rule documents (with metadata) into a ChromaDB collection using the Gemini Flash embedding API in `retrieval_document` mode.

2. **Retrieval:**  
   Given a user query, retrieve multiple relevant passages from the ChromaDB collection by switching to `retrieval_query` mode.

3. **Generation:**  
   Combine the retrieved passages with the user query into an enhanced prompt (including few-shot examples) and generate an answer using the Gemini Flash API. An iterative feedback loop refines the answer for clarity and detail.

4. **Interactive Interface:**  
   Use **ipywidgets** to build an interactive Q&A interface directly in your Jupyter Notebook for live querying and answer generation.

5. **Caching & Modular Enhancements:**  
   Utilize an LRU cache to reduce redundant API calls for repeated prompts.

In [31]:
!pip uninstall -qqy jupyterlab kfp 
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip install streamlit
!pip install ipywidgets

In [13]:
# %%
from google import genai
from google.genai import types
from IPython.display import Markdown

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

In [14]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [16]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

In [17]:
# %% [markdown]
# ## Enhanced Data Indexing with Metadata
# In this section, we create several board game rule documents with metadata. We then index them into a ChromaDB collection.


In [18]:
# %%
# Sample documents with additional metadata
documents = [
    {
        "id": "doc1",
        "title": "Rules Overview - Basic Setup",
        "text": (
            "Empire Builders Rules - Document 1: Setup: Each player starts with 100 gold and 5 units. "
            "Objective: Conquer territories on a hexagonal map. Turn order: Players alternate turns, beginning with the youngest player."
        ),
        "edition": "v1",
        "source": "Rulebook v1"
    },
    {
        "id": "doc2",
        "title": "Rules Detail - Movement & Combat",
        "text": (
            "Empire Builders Rules - Document 2: Movement & Combat: In your turn, you can move units up to 3 spaces. "
            "Combat is resolved by comparing unit strengths, with some randomness. Special mechanics enhance conflict dynamics."
        ),
        "edition": "v1",
        "source": "Rulebook v1"
    },
    {
        "id": "doc3",
        "title": "Rules - Special Conditions",
        "text": (
            "Empire Builders Rules - Document 3: Special Rules: If a player occupies an opponent's territory for two consecutive turns, "
            "that opponent loses control. The game ends when only one player remains."
        ),
        "edition": "v1",
        "source": "Rulebook v1"
    },
    {
        "id": "doc4",
        "title": "Advanced Tactics",
        "text": (
            "Empire Builders Advanced Tactics: Advanced players use formation strategies and resource allocation to outmaneuver opponents."
        ),
        "edition": "v2",
        "source": "Expert Guide"
    },
]

# Split documents into text, metadata, and IDs.
doc_texts = [doc["text"] for doc in documents]
doc_metadatas = [{"title": doc["title"], "edition": doc["edition"], "source": doc["source"]} for doc in documents]
doc_ids = [doc["id"] for doc in documents]

# Set up your embedding function and ChromaDB collection as before:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google import genai
from google.genai import types

# Ensure you have your API key set up as previously.
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

# Define a helper to retry when hitting rate limits.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents or queries.
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        embedding_task = "retrieval_document" if self.document_mode else "retrieval_query"
        response = client.models.embed_content(
            model="models/text-embedding-004",  # or "models/gemini-embedding-exp-03-07"
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task),
        )
        return [e.values for e in response.embeddings]

# Create or get the collection.
import chromadb
chroma_client = chromadb.Client()
DB_NAME = "empirebuildersdb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Index the documents with their metadata.
db.add(documents=doc_texts, metadatas=doc_metadatas, ids=doc_ids)

print("Documents in DB:", db.count())


Documents in DB: 4


In [19]:
# %% [markdown]
# ## Multi-Passage Retrieval & Enhanced Prompt Engineering
# We now retrieve multiple passages (here, using n_results=3) and construct a detailed generation prompt 
# incorporating few-shot examples.

In [21]:
# %%
# Switch the embedding function to query mode.
embed_fn.document_mode = False

# Define a sample query.
query = "How do players initiate the game setup?"

# Retrieve multiple passages.
result = db.query(query_texts=[query], n_results=3)
retrieved_passages = result["documents"][0]  # List of passages
print("Retrieved Passages:")
for passage in retrieved_passages:
    print("-", passage.replace("\n", " "))

# Define few-shot examples to guide generation.
few_shot_examples = (
    "Example 1:\n"
    "Q: How do players start the game?\n"
    "A: Each player begins with a fixed amount of gold and units, and the game commences with turn-based actions.\n\n"
    "Example 2:\n"
    "Q: What happens if an opponent's territory is occupied?\n"
    "A: If a player occupies an opponent's territory for two consecutive turns, control is lost."
)

def create_enhanced_generation_prompt(query, passages, few_shot=None):
    prompt = (
        "You are an expert board game rules interpreter. Answer the following question by synthesizing the provided "
        "passages. Use the few-shot examples as guidance for clarity and detail.\n\n"
    )
    if few_shot:
        prompt += "Few-shot Examples:\n" + few_shot + "\n\n"
    prompt += f"QUESTION: {query}\n\n"
    prompt += "PASSAGES:\n"
    for passage in passages:
        prompt += passage.replace("\n", " ") + "\n"
    prompt += "\nAnswer:"
    return prompt

# Create the prompt.
generation_prompt = create_enhanced_generation_prompt(query, retrieved_passages, few_shot_examples)
print("\n Generation Prompt:\n", generation_prompt)


Retrieved Passages:
- Empire Builders Rules - Document 1: Setup: Each player starts with 100 gold and 5 units. Objective: Conquer territories on a hexagonal map. Turn order: Players alternate turns, beginning with the youngest player.
- Empire Builders Rules - Document 2: Movement & Combat: In your turn, you can move units up to 3 spaces. Combat is resolved by comparing unit strengths, with some randomness. Special mechanics enhance conflict dynamics.
- Empire Builders Advanced Tactics: Advanced players use formation strategies and resource allocation to outmaneuver opponents.

 Generation Prompt:
 You are an expert board game rules interpreter. Answer the following question by synthesizing the provided passages. Use the few-shot examples as guidance for clarity and detail.

Few-shot Examples:
Example 1:
Q: How do players start the game?
A: Each player begins with a fixed amount of gold and units, and the game commences with turn-based actions.

Example 2:
Q: What happens if an opponen

In [22]:
# %% [markdown]
# ## Iterative Feedback Loop
# We simulate an extra refinement step where the initial answer is improved for clarity and completeness.

In [23]:
def generate_answer(prompt_text):
    """Generate an answer from the Gemini Flash model using the actual API call."""
    response = client.models.generate_content(
         model="gemini-2.0-flash",  # Use your desired Gemini Flash model version.
         contents=prompt_text
    )
    return response.text

# Generate the initial answer using an enhanced prompt (generation_prompt defined earlier).
initial_answer = generate_answer(generation_prompt)
print("Initial Answer:\n", initial_answer)

def refine_answer(initial_answer, query, passages):
    """
    Refine the initial answer using the Gemini Flash API.
    This function creates a refinement prompt that instructs the model to review and improve the initial response.
    """
    refinement_prompt = (
        "Review the following answer and refine it so that it is more clear, detailed, "
        "and accessible to a non-technical audience. Include all relevant details from the provided passages if needed.\n\n"
        f"Initial Answer: {initial_answer}\n"
        f"QUESTION: {query}\n"
        "PASSAGES: " + " ".join(passages) + "\n\nRefined Answer:"
    )
    response = client.models.generate_content(
         model="gemini-2.0-flash",
         contents=refinement_prompt
    )
    refined = response.text
    return refined

# Assume 'query' is defined and 'retrieved_passages' is a list of passages retrieved earlier.
refined_answer = refine_answer(initial_answer, query, retrieved_passages)
print("\nRefined Answer:\n", refined_answer)


Initial Answer:
 Each player starts with 100 gold and 5 units to begin the game.


Refined Answer:
 Here's a refined answer, aimed at being clear, detailed, and accessible to a non-technical audience, drawing from all provided passages:

**To start a game of Empire Builders, each player begins with two things:**

*   **Resources:** Every player receives 100 gold to use for various actions in the game, like recruiting more units or upgrading their territories (though the document doesn't explicitly state gold use, it is implied by mentioning "resource allocation" in advanced tactics).
*   **Forces:** Each player starts with a small army of 5 units. These units will be used to explore the map, claim territories, and engage in combat with other players.

The game is played on a hexagonal map where players try to conquer territories. The youngest player takes the first turn, and then players alternate turns in a clockwise order.

**In summary, the initial setup focuses on giving each playe

In [24]:
# %% [markdown]
# ## Interactive Q&A with Streamlit (not possible on notebook execution)
# The following is an interactive Q&A interface.

In [30]:
# %%
# Save this as rag.py and run with "streamlit run rag.py"
import streamlit as st

st.title("Board Game Rules Q&A")
query_input = st.text_input("Enter your question:")

if st.button("Get Answer"):
    # Retrieve passages from Chroma DB.
    embed_fn.document_mode = False
    result = db.query(query_texts=[query_input], n_results=3)
    retrieved = result["documents"][0]
    
    # Create enhanced prompt.
    gen_prompt = create_enhanced_generation_prompt(query_input, retrieved, few_shot_examples)
    
    # Generate answer and refine.
    init_ans = generate_answer(gen_prompt)
    final_ans = refine_answer(init_ans, query_input, retrieved)
    
    st.write("**Final Answer:**")
    st.write(final_ans)


2025-04-15 00:45:57.151 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.216 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-15 00:45:57.216 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.218 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.219 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-15 00:45:57.220 Session state does not 

In [33]:
# Using ipywidgets for interactive session on notebooks

import ipywidgets as widgets
from IPython.display import display, clear_output

def retrieve_passages_from_chroma(query: str, n_results: int = 3):
    """
    Query the ChromaDB collection for relevant documents based on the query.
    Assumes the embedding function is set to query mode.
    
    Parameters:
      query (str): The user's query.
      n_results (int): Number of top passages to retrieve.
    
    Returns:
      List of retrieved passages (strings).
    """
    # Ensure the embedding function is in query mode.
    embed_fn.document_mode = False
    result = db.query(query_texts=[query], n_results=n_results)
    # result["documents"] is a list, where the first element is a list of retrieved passages.
    passages = result["documents"][0]
    return passages

# -------------------- Enhanced Prompt Creation --------------------
def create_enhanced_generation_prompt(query: str, passages: list, few_shot: str = None) -> str:
    """
    Create an enhanced generation prompt that synthesizes the query, few-shot examples,
    and the retrieved passages.
    
    Parameters:
      query (str): The user's query.
      passages (list): List of retrieved document passages.
      few_shot (str): Optional few-shot examples to include.
      
    Returns:
      A complete prompt string.
    """
    prompt = (
        "You are an expert board game rules interpreter. "
        "Synthesize the following passages to answer the question comprehensively in a clear, conversational tone.\n\n"
    )
    if few_shot:
        prompt += "Few-shot Examples:\n" + few_shot + "\n\n"
    prompt += f"QUESTION: {query}\n\n"
    prompt += "PASSAGES:\n"
    for passage in passages:
        prompt += passage.replace("\n", " ") + "\n"
    prompt += "\nAnswer:"
    return prompt

# -------------------- Actual Answer Generation --------------------
def generate_answer(prompt_text: str) -> str:
    """
    Generate an answer from the Gemini Flash API using your actual API call.
    
    Parameters:
      prompt_text (str): The complete generation prompt.
      
    Returns:
      The generated answer as a string.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",  # Replace with your desired Gemini Flash model version.
        contents=prompt_text
    )
    return response.text

def refine_answer(initial_answer: str, query: str, passages: list) -> str:
    """
    Refine the initial answer to be more clear, detailed, and accessible.
    
    Parameters:
      initial_answer (str): The first generated answer.
      query (str): The original query.
      passages (list): The retrieved passages.
      
    Returns:
      A refined answer as a string.
    """
    refinement_prompt = (
        "Review the following answer and refine it so that it is more clear, detailed, "
        "and accessible to a non-technical audience. Include all relevant details from the provided passages if needed.\n\n"
        f"Initial Answer: {initial_answer}\n"
        f"QUESTION: {query}\n"
        "PASSAGES: " + " ".join(passages) + "\n\nRefined Answer:"
    )
    response = client.models.generate_content(
         model="gemini-2.0-flash",
         contents=refinement_prompt
    )
    return response.text

# -------------------- Few-Shot Examples --------------------
few_shot_examples = (
    "Example 1:\n"
    "Q: How do players start the game?\n"
    "A: Each player begins with a fixed amount of gold and units, and the game commences with turn-based actions.\n\n"
    "Example 2:\n"
    "Q: What occurs when an opponent's territory is occupied for multiple turns?\n"
    "A: Consecutive occupation of an opponent's territory results in special rules that may cause the opponent to lose control."
)

# -------------------- ipywidgets Interactive Interface --------------------
# Create the text input widget for user's question.
question_input = widgets.Text(
    value='',
    placeholder='Enter your board game question here...',
    description='Question:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

# Create a button to trigger answer generation.
generate_button = widgets.Button(
    description='Get Answer',
    disabled=False,
    button_style='success',
    tooltip='Click to generate answer',
    icon='check'
)

# Create an output area widget to display the results.
output_area = widgets.Output()

def on_generate_button_clicked(b):
    with output_area:
        clear_output()  # Clear previous output.
        query = question_input.value.strip()
        if not query:
            print("Please enter a question.")
            return
        
        # Retrieve passages from the actual ChromaDB.
        passages = retrieve_passages_from_chroma(query, n_results=3)
        
        # Create an enhanced generation prompt.
        prompt_text = create_enhanced_generation_prompt(query, passages, few_shot_examples)
        
        # Generate the initial answer using the Gemini Flash API.
        initial_ans = generate_answer(prompt_text)
        
        # Refine the answer for improved clarity.
        final_ans = refine_answer(initial_ans, query, passages)
        
        # Display both the generation prompt and the final answer.
        print("Generation Prompt:")
        print(prompt_text)
        print("\nFinal Answer:")
        print(final_ans)

# Connect the button click event with the function.
generate_button.on_click(on_generate_button_clicked)

# Display the interactive widgets.
display(question_input, generate_button, output_area)


Text(value='', description='Question:', layout=Layout(width='80%'), placeholder='Enter your board game questio…

Button(button_style='success', description='Get Answer', icon='check', style=ButtonStyle(), tooltip='Click to …

Output()

In [ ]:
# %% [markdown]
# ## Caching Results to Improve Performance
# We use Python's LRU cache to store recent API call results, reducing redundant requests.


In [34]:
from functools import lru_cache

@lru_cache(maxsize=32)
def cached_generate_answer(prompt_text: str) -> str:
    """
    Generate an answer using the Gemini Flash API call and cache the result.
    Using caching can reduce redundant calls for the same prompt.
    
    Parameters:
    - prompt_text: The text of the prompt provided to the Gemini Flash API.
    
    Returns:
    - The answer text as returned by the Gemini Flash model.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",  # Replace with the desired Gemini Flash model version.
        contents=prompt_text
    )
    return response.text

# Example usage:
cached_answer = cached_generate_answer(generation_prompt)
print("Cached Answer:\n", cached_answer)


Cached Answer:
 The game begins with each player possessing 100 gold and 5 units. Players then alternate turns, starting with the youngest player, to conquer territories on a hexagonal map.

